In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
#algoritmo principale
def DTWDistance(signalA, signalB):
    n,m = len(signalA), len(signalB)
    DTW = np.zeros(shape=(n, m))
    for i in range(0,n):
        DTW[i,0] = np.finfo(dtype=float).max
    for i in range(0,m):
        DTW[0,i] = np.finfo(dtype=float).max
    DTW[0,0] = 0
    
    for i in range(1,n):
        for j in range(1,m):
            const = abs(signalA[i] - signalB[j])
            DTW[i,j] = const + min(DTW[i-1, j], DTW[i, j-1], DTW[i-1, j-1])
    
    return DTW[n - 1, m - 1]

In [5]:
#Algoritmo di scaling
def new_scale_df(data_frame, gyro, accl, pres):
    
    def scale(value, max_, min_):
        value -= min_
        value /= max_ - min_
    
    copy = data_frame.copy() 
    scale(copy.AcX, accl, -accl)
    scale(copy.AcY, accl, -accl)
    scale(copy.AcZ, accl, -accl)
    scale(copy.GyX, gyro, -gyro)
    scale(copy.GyY, gyro, -gyro)
    scale(copy.GyZ, gyro, -gyro)
    scale(copy.Pres, pres, 0)

    return copy

In [ ]:
with open('../provaLSTM/giovanni.json', 'r') as f:
    giovanni_data = json.load(f)


